In [1]:
import gensim
from xlrd import open_workbook
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import pandas as pd
from nltk.corpus import stopwords

import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import json
from gensim import similarities
import numpy as np

In [2]:
"""
book = open_workbook('UNSPSC English v220601 project.xlsx')
'''book = open_workbook('Unspec List2b.xlsx')'''
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

print(len(dict_list))
    
doclist =[]
listOfEntry =[]
entrylist = []
df7 = pd.DataFrame()


for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    elif(entry.get("Family")!="" and entry.get("Class")==""):
        if(listOfEntry!=[]):
            #print(listOfEntry)
            doclist += entrylist
            continue
    elif(entry.get("Commodity")==""):
        continue
    else:
        if(entry.get("Class Definition")!=""):
            f = str(sent_tokenize(entry.get("Class Definition").lower())).strip('[]').strip('\'')
            #print(f)
        else:
            f=str([" "]).strip('[]').strip('\'')
        e = str(sent_tokenize(entry.get("Class Title").lower())).strip('[]').strip('\'')
        #print(e)
        g = str(sent_tokenize(entry.get("Commodity Title").lower())).strip('[]').strip('\'')
        #print(g)
        if(entry.get("Commodity Definition")!=""):
            h = str(sent_tokenize(entry.get("Commodity Definition").lower())).strip('[]').strip('\'')
            #print(h)
        else:
            h = str([" "]).strip('[]').strip('\'')
        i=int(entry.get("Commodity"))
        df7 = df7.append({'class':e+f,'commodity':g+h,'level_1': i},ignore_index=True)
        
  
        
doclist.append(listOfEntry) 

print("done")        
"""        
unspec_df = pd.read_excel('UNSPSC English v220601 project.xlsx','Sheet1', index_col=None, na_values=['NA'])
unspec = unspec_df.drop(['Family Definition','Family','Family Title','Segment Title','Segment Definition','Segment','Version','Key','Synonym','Acronym'],axis = 1)
cols = ['Class Title','Class Definition']
cols2 =['Commodity Title','Commodity Definition']
unspec["class"] = unspec[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unspec["commodity"] = unspec[cols2].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unspec2 = unspec.drop(['Class','Class Title','Class Definition','Commodity Title','Commodity Definition'],axis=1)
unspec2=unspec2.rename(columns={"Commodity":"level_1"})


In [3]:
county_df = pd.read_excel('eCAPS_COMM_11072019.xlsx','COMM_CLS', index_col=None, na_values=['NA'])
county_df2 = pd.read_excel('eCAPS_COMM_11072019.xlsx','COMM_ITM', index_col=None, na_values=['NA'])
county_df.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
county_df2.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
countyresult = pd.merge(county_df,county_df2,on='COMM_CLS')
cr = countyresult.drop(['DSCR_EXT_y','DSCR_EXT_DV_x','DSCR_EXT_x','COMM_DSCR_UP_x','COMM_CD_x','COMM_ITM_x','COMM_GP_x','COMM_DET_x','ACT_INACT_FL_x','NAICS_CD_x','COMM_DET_y','COMM_GP_y','COMM_DET_y','ACT_INACT_FL_y','NAICS_CD_y','COMM_DSCR_UP_y','DSCR_EXT_DV_y'],axis=1)
cr2 = cr.rename(columns={"KEYWD_x":"class","KEYWD_y":"commodity","COMM_CD_y":"level_1"})


In [4]:

df1= cr2[['class','commodity','level_1']]
df2 = df1.append(unspec2,ignore_index=True,sort=False)


In [5]:
df2.head()

,class,commodity,level_1
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADIO-NAVIGATION-AIRCRAFT,3510
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADAR-AIRCRAFT,3515
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AIRPLANES,3520
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AUTOMATIC PILOT SYSTEMS,3525
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",BAGGAGE HANDLING EQUIPMENT AND PARTS,3526


In [6]:
df9 = df2.apply(lambda x: ','.join(x.astype(str)),axis = 1)

In [7]:
df_clean2 = pd.DataFrame({'clean':df9})

In [8]:
sent23 = [row.split(',') for row in df_clean2['clean']]

In [9]:
print(len(sent23))

163115


In [10]:
model2 = gensim.models.Word2Vec(sent23,min_count=1,size=300, workers=6, window= 5,sg=1)

In [11]:
model2.wv.__getitem__('39101601')

array([-6.85129641e-03, -2.76205334e-04, -1.12283807e-02,  3.37327644e-03,
        3.47680482e-03, -9.92406812e-03, -7.40479399e-03, -3.67545942e-03,
       -9.20144934e-03,  8.66645202e-03,  9.77519993e-03, -1.56449601e-02,
       -3.39640700e-03, -1.85167836e-03, -3.23084160e-03,  1.52392760e-02,
       -6.99438853e-03,  6.38173660e-04,  6.86117588e-03, -3.25707806e-04,
       -1.01559628e-02,  7.68182892e-03, -6.34609163e-03,  5.36206504e-03,
       -1.48594929e-02,  5.11966925e-03, -7.74337503e-04, -1.26256084e-03,
       -2.06868513e-03,  7.62484968e-03,  6.03216747e-03,  7.48218503e-03,
        1.97902285e-02, -2.87198252e-03, -4.34847490e-04,  9.09088179e-03,
       -1.13778724e-03,  2.79324631e-05, -1.28321617e-03, -2.59092194e-03,
        4.11928204e-06, -9.27977730e-03,  1.10031534e-02, -5.05001005e-03,
       -3.03095137e-03, -3.44610563e-03,  1.41873548e-03, -1.39590828e-02,
        2.93532223e-03, -2.17752066e-03,  1.66607974e-03, -6.98467158e-03,
        2.33955518e-03,  

In [12]:
model2.wv.similarity('39101601','39101603')

0.8112489

In [16]:
model2.wv.similarity('39101601','3105')

0.9365732

In [14]:
model2.wv.most_similar('3105')[:15]

[(' INDUSTRIAL TYPE', 0.9911402463912964),
 ('FILTERS AND FILTER MEDIA', 0.991096019744873),
 ('3109', 0.9903091192245483),
 (' VENTED AND UNVENTED) AND PARTS', 0.9901311993598938),
 (' AIR CONDITIONING: HERMETICALLY SEALED', 0.9899585843086243),
 (' COOLING TOWER WATER TREATMENT: PH', 0.9892481565475464),
 ('GRILLES', 0.988997757434845),
 ('AIR CONDITIONERS: CONTROLLED ENVIRONMENT (FOR COMPUTER ROOMS',
  0.9889377355575562),
 (' AND PARTS AND ACCESSORIES NOT INDIVIDUALLY ITEMIZED)', 0.9889260530471802),
 ('FILTER COATING', 0.9889007806777954)]

In [15]:
"""
model2.wv.vocab
model2.save("W2V-Model")

text = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"
idt =['101'] 
text = text.lower()
newEntry = []
newEn =[]
stop_words = set(stopwords.words('english'))
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]


Wtest = word_tokenize(text)
Wtest = [ j for j in Wtest if not j in ch ]
Wtest = [w for w in Wtest if not w in stop_words]
newEntry.append(Wtest)

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element+" ")
    return result

newEntry[0] = concatenate_list_data(newEntry[0])
newEntry += idt
newEn.append(newEntry)
print(newEn)

new_mod = gensim.models.Word2Vec.load("W2V-Model")
new_mod.build_vocab(newEn,update = True)
new_mod.train(newEn,total_examples=1,epochs = 1)

new_mod.wv.similarity('30121703.0','101')
new_mod.wv.most_similar_cosmul('101')

new_mod.wv.similar_by_vector('101')

"""

'\nmodel2.wv.vocab\nmodel2.save("W2V-Model")\n\ntext = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"\nidt =[\'101\'] \ntext = text.lower()\nnewEntry = []\nnewEn =[]\nstop_words = set(stopwords.words(\'english\'))\nch = [ \'``\',\'``\', "\'\'",\',\',\'.\',\'\\n\',"\'",";",":","(",")","-","--"]\n\n\nWtest = word_tokenize(text)\nWtest = [ j for j in Wtest if not j in ch ]\nWtest = [w for w in Wtest if not w in stop_words]\nnewEntry.append(Wtest)\n\ndef concatenate_list_data(list):\n    result= \'\'\n    for element in list:\n        result += str(element+" ")\n    return result\n\nnewEntry[0] = concatenate_list_data(newEntry[0])\nnewEntry += idt\nnewEn.append(newEntry)\nprint(newEn)\n\nnew_mod = gensim.models.Word2Vec.load("W2V-Model")\nnew_mod.build_vocab(newEn,update = True)\nnew_mod.train(newEn,total_examples=1,epochs = 1)\n\nnew_mod.wv.similarity(\'30121703.0\',\'101\')\nnew_mod.wv.most_similar_cosmul(\'101\')\n\nnew_mod.wv.similar_by_vector(\'10